### Homological Product Code Construction
* Can view quantum CSS codes as chain complexes
$$C_2 \rightarrow C_1 \rightarrow C_0$$
* Can take tensor products of chain complexes (CSS codes) as per the construction given by Hastings et. al.
* There are two 'theories' when it comes to defining CSS codes in terms of homology theory
    1. 'Single sector' theory - Use a single vector space to define chain complex (Will try this first)
    2. 'Multiple sector' theory - Use 3 vector spaces to define chain complex
* $\text{\underline{1st Test: Utilizing Self-Orthogonal Codes}}$
    * Use 2 $[[7,1,3]]$ codes

In [2]:
from sage.all import *
import numpy as np
from typing import List

In [3]:
def construct_hom_classical_code(pc_mat: np.array) -> ChainComplex:
    """ 
    Construct chain complex for classical code from parity-check matrix

    Parameters:
    -----------
    pc_mat - parity-check matrix for code

    Returns:
    --------
    Chain complex representing code
    """
    # Define vector spaces for chain complex
    k_1 = len(pc_mat)
    num_bits = len(pc_mat[0])
    C_0 = VectorSpace(GF(2), num_bits)
    C_1 = VectorSpace(GF(2), k_1)

    # Define boundary operators
    d_0 = matrix(pc_mat.tolist())

    # Define chain complex
    chain_complex_data = [C_0, d_0, C_1]
    #chain_complex_data = {1:d_1}
    code = ChainComplex(data=chain_complex_data, base_ring=GF(2), degree_of_differential=1)
    return code

def construct_hom_CSS_code(pc_mat_X: np.array, pc_mat_Z: np.array) -> ChainComplex:
    """ 
    Construct chain complex for CSS code from parity-check matrices

    Parameters:
    -----------
    pc_mat_X - X parity-check matrix
    pc_mat_Z - Z parity-check matrix

    Returns:
    --------
    Chain complex representing CSS code
    """

    # Define vector spaces for chain complex
    k_1 = len(pc_mat_Z)
    k_2 = len(pc_mat_X)
    num_qubits = len(pc_mat_Z[0])
    C_0 = VectorSpace(GF(2), k_1)
    C_1 = VectorSpace(GF(2), num_qubits)
    C_2 = VectorSpace(GF(2), k_2)

    # Define boundary operators
    d_0 = matrix((pc_mat_Z.T).tolist())
    d_1 = matrix(pc_mat_X.tolist())

    # Define chain complex
    chain_complex_data = [C_0, d_0, C_1, d_1, C_2]
    #chain_complex_data = {1:d_1, 2:d_2}
    code = ChainComplex(data=chain_complex_data, base_ring=GF(2), degree_of_differential=1)
    return code

def construct_hom_prod_code(c_1: ChainComplex, c_2: ChainComplex) -> ChainComplex:
    """ 
    Construct homological product code by taking tensor product of chain complexes 
    corresponding to quantum CSS/classical codes

    Parameters:
    -----------
    c_1 - First code
    c_2 - Second code

    Returns:
    --------
    Chain complex corresponding to homological product code
    """
    return c_1.tensor(c_2)

def compute_logical_Z_ops(c: ChainComplex) -> List:
    """ 
    Return logical Z operators for quantum code corresponding to chain complex 'c'

    Parameters:
    -----------
    c - Quantum code

    Returns:
    --------
    Logical operators explicitly as list.
    '0' -> Identity
    '1' -> Pauli Z operator
    """
    # Relevant Boundary operators
    d_0 = c.differential(0)
    d_0_T = d_0.transpose()
    d_1 = c.differential(1)
    d_1_T = d_1.transpose() 

    # Kernel and Image of boundary operators to compute logial operators
    ker_d_1 = d_1.right_kernel()
    im_d_0 = d_0.transpose().image()

    # Logical Z operators
    ker_d_1_vecs = [v for v in ker_d_1]
    im_d_0_vecs = [v for v in im_d_0]
    log_Z_ops = [v for v in ker_d_1_vecs if v not in im_d_0_vecs]

    return log_Z_ops

def compute_logical_Z_ops_v2(H_a: np.array, H_b: np.array) -> List: 
    """
    Return logical Z operators for quantum code corresponding to
    hypergraph product code constructed from H_a and H_b

    Params:
    -------
    H_a - First classical code
    H_b - Second classical code 

    Returns:
    --------
    Logical Z operators
    """
    H_a = matrix(H_a.tolist())
    H_b = matrix(H_b.tolist())
    H_b_T = H_b.transpose()

    ker_H_a = H_a.right_kernel()
    im_H_b_T_comp = H_b_T.image().complement()
    ker_H_b_T = H_b_T.right_kernel()
    im_H_a_comp = H_a.image().complement()

    




def compute_logical_X_ops(c: ChainComplex) -> List:
    """ 
    Return logical X operators for quantum code corresponding to chain complex 'c'

    Parameters:
    -----------
    c - Quantum code

    Returns:
    --------
    Logical operators explicitly as list.
    '0' -> Identity
    '1' -> Pauli X operator
    """
    # Relevant Boundary operators
    d_0 = c.differential(0)
    d_0_T = d_0.transpose()
    d_1 = c.differential(1)
    d_1_T = d_1.transpose() 

    # Kernel and Image of boundary operators to compute logial operators
    im_d_1_T = d_1_T.transpose().image()
    ker_d_0_T = d_0.right_kernel()

    # Logical X operators
    ker_d_0_T_vecs = [v for v in ker_d_0_T]
    im_d_1_T_vecs = [v for v in im_d_1_T]
    log_X_ops = [v for v in ker_d_0_T_vecs if v not in im_d_1_T_vecs]
    return log_X_ops

In [4]:
# Let us begin by defining our chain complex (single sector theory) for Steane Code
x_pc_mat = np.array([[0,0,0,1,1,1,1], [0,1,1,0,0,1,1,], [1,0,1,0,1,0,1]])
z_pc_mat = np.array([[0,0,0,1,1,1,1], [0,1,1,0,0,1,1,], [1,0,1,0,1,0,1]])

n = 7 # Number of physical qubits in CSS code

steane_code_1 = construct_hom_CSS_code(x_pc_mat, z_pc_mat)
steane_code_2 = construct_hom_CSS_code(x_pc_mat, z_pc_mat)
hom_prod_code = construct_hom_prod_code(steane_code_1, steane_code_2)
ascii_art(hom_prod_code)


/home/samyaknn/anaconda3/envs/qc_env/lib/python3.9/site-packages/sage/typeset/character_art.py:400: UserWarning: the console size is smaller than the pretty representation of the object
  warnings.warn("the console size is smaller than the pretty "


           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
 0 <-- C_4 

                                                                                       
                                                                                       
       

In [7]:
pc_mat = np.array([[1,1,0],[0,1,1],[1,0,1]])
rep_code = construct_hom_classical_code(pc_mat)
toric_code = construct_hom_prod_code(rep_code, rep_code)
#print(toric_code.homology(generators=True))
#print(ascii_art(toric_code))
#X_log_ops = compute_logical_X_ops(toric_code)
#Z_log_ops = compute_logical_Z_ops(toric_code)
#print(X_log_ops)
#print(Z_log_ops)
print(rank(toric_code.differential(0)), rank(toric_code.differential(1)))

                                                        [1 1 0 0 0 0 0 0 0] 
                                                        [0 1 1 0 0 0 0 0 0] 
                                                        [1 0 1 0 0 0 0 0 0] 
                                                        [0 0 0 1 1 0 0 0 0] 
                                                        [0 0 0 0 1 1 0 0 0] 
                                                        [0 0 0 1 0 1 0 0 0] 
                                                        [0 0 0 0 0 0 1 1 0] 
                                                        [0 0 0 0 0 0 0 1 1] 
                                                        [0 0 0 0 0 0 1 0 1] 
            [1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0]       [1 0 0 1 0 0 0 0 0] 
            [0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0]       [0 1 0 0 1 0 0 0 0] 
            [0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0]       [0 0 1 0 0 1 0 0 0] 
            [0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0]       [0 0 0 1 0 0 1 0 0] 

In [6]:
min(Z_log_ops)

NameError: name 'Z_log_ops' is not defined

In [ ]:
type(rep_code.homology(0))

sage.modules.free_module.FreeModule_ambient_field_with_category